In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
dataset = pd.read_csv('../HW1_data/traffic/Casualties0514.csv')
dataset = dataset.drop(["Accident_Index", "Vehicle_Reference", "Casualty_Reference"], axis=1)
dataset.head()

,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type
0,3,1,37,7,2,1,1,0,0,-1,0,1
1,2,1,37,7,3,0,0,0,4,-1,11,1
2,1,1,62,9,3,0,0,0,0,-1,9,1
3,3,1,30,6,3,5,2,0,0,-1,0,1
4,1,1,49,8,3,0,0,0,0,-1,3,-1


In [41]:
# Prepare data for Decision Tree
from sklearn.model_selection import train_test_split
X = dataset.drop(["Casualty_Severity"], axis=1)
y = dataset["Casualty_Severity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6357)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=6357)
del X, y
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1418700, 11)
(354676, 11)
(443344, 11)


In [47]:
from sklearn.tree import DecisionTreeClassifier
import pydotplus as pydot
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn import tree

# Vanilla Decision Tree
dt = DecisionTreeClassifier(criterion="entropy", max_depth=10, random_state=4321)
dt.fit(X_train, y_train)
print("Fit end! ")
dt_val_score = dt.score(X_val, y_val)
print("Validation score: ", dt_val_score)

Fit end! 
Validation score:  0.8773190179205811


In [48]:
# Post-prune Decision Tree 
# My own implementation based on https://stackoverflow.com/questions/49428469/pruning-decision-trees/49496027#49496027
# Pruning based on validation score (if pruning a subtree gains better performance on validate set, then prune it)
from sklearn.tree._tree import TREE_LEAF

def prune_from_root(inner_tree, index):
    global dt_val_score
    memo = (inner_tree.children_left[index], inner_tree.children_right[index])
    inner_tree.children_left[index] = TREE_LEAF
    inner_tree.children_right[index] = TREE_LEAF
    new_score = dt.score(X_val, y_val)
    if new_score <= dt_val_score:
        inner_tree.children_left[index] = memo[0]
        inner_tree.children_right[index] = memo[1]
    else:
        dt_val_score = new_score

    # if there are children, visit them as well
    if inner_tree.children_left[index] != TREE_LEAF:
        prune_from_root(inner_tree, inner_tree.children_left[index])
        prune_from_root(inner_tree, inner_tree.children_right[index])

def prune_from_leaf(inner_tree, index):
    global dt_val_score
    # if there are children, firstly visit them
    if inner_tree.children_left[index] != TREE_LEAF:
        prune_from_leaf(inner_tree, inner_tree.children_left[index])
        prune_from_leaf(inner_tree, inner_tree.children_right[index])
        memo = (inner_tree.children_left[index], inner_tree.children_right[index])
        inner_tree.children_left[index] = TREE_LEAF
        inner_tree.children_right[index] = TREE_LEAF
        new_score = dt.score(X_val, y_val)
        if new_score <= dt_val_score:
            inner_tree.children_left[index] = memo[0]
            inner_tree.children_right[index] = memo[1]
        else:
            dt_val_score = new_score

In [49]:
# prune from root
dt = DecisionTreeClassifier(criterion="entropy", random_state=4321)
dt.fit(X_train, y_train)
dt_val_score = dt.score(X_val, y_val)
print("no-prune validation score: ", dt_val_score)
print("no-prune test score: ", dt.score(X_test, y_test))
prune_from_root(dt.tree_, 0)
print("post-prune from root validation score: ", dt.score(X_val, y_val))
print("post-prune from root test score: ", dt.score(X_test, y_test))

no-prune validation score:  0.8692243061272824
no-prune test score:  0.8686212061063192
post-prune from root validation score:  0.8774712695530569
post-prune from root test score:  0.8767503338265545


In [50]:
# prune from leaf
# create an identical decision tree to the previous one using the same random_state
dt = DecisionTreeClassifier(criterion="entropy", random_state=4321)
dt.fit(X_train, y_train)
dt_val_score = dt.score(X_val, y_val)
print("no-prune validation score: ", dt_val_score)
print("no-prune test score: ", dt.score(X_test, y_test))
prune_from_leaf(dt.tree_, 0)
print("post-prune from leaf validation score: ", dt.score(X_val, y_val))
print("post-prune from leaf test score: ", dt.score(X_test, y_test))

no-prune validation score:  0.8692243061272824
no-prune test score:  0.8686212061063192


KeyboardInterrupt: 

In [31]:
for col in ["Casualty_Class", "Sex_of_Casualty", "Age_Band_of_Casualty", "Pedestrian_Location", "Pedestrian_Movement", "Car_Passenger", "Bus_or_Coach_Passenger", "Pedestrian_Road_Maintenance_Worker", "Casualty_Type", "Casualty_Home_Area_Type"]: 
    need_remove_non = False
    if dataset[col].min() < 0:
        need_remove_non = True
    dataset = pd.concat([dataset,pd.get_dummies(dataset[col], prefix=col)],axis=1).drop([col],axis=1)
    if need_remove_non:
        dataset = dataset.drop([col+"_-1"], axis=1)
dataset.head()

,Age_of_Casualty,Casualty_Severity,Casualty_Class_1,Casualty_Class_2,Casualty_Class_3,Sex_of_Casualty_1,Sex_of_Casualty_2,Age_Band_of_Casualty_1,Age_Band_of_Casualty_2,Age_Band_of_Casualty_3,...,Casualty_Type_20,Casualty_Type_21,Casualty_Type_22,Casualty_Type_23,Casualty_Type_90,Casualty_Type_97,Casualty_Type_98,Casualty_Home_Area_Type_1,Casualty_Home_Area_Type_2,Casualty_Home_Area_Type_3
0,37,2,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,37,3,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,62,3,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,30,3,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,49,3,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
dataset.columns

Index(['Age_of_Casualty', 'Casualty_Severity', 'Casualty_Class_1',
       'Casualty_Class_2', 'Casualty_Class_3', 'Sex_of_Casualty_1',
       'Sex_of_Casualty_2', 'Age_Band_of_Casualty_1', 'Age_Band_of_Casualty_2',
       'Age_Band_of_Casualty_3', 'Age_Band_of_Casualty_4',
       'Age_Band_of_Casualty_5', 'Age_Band_of_Casualty_6',
       'Age_Band_of_Casualty_7', 'Age_Band_of_Casualty_8',
       'Age_Band_of_Casualty_9', 'Age_Band_of_Casualty_10',
       'Age_Band_of_Casualty_11', 'Pedestrian_Location_0',
       'Pedestrian_Location_1', 'Pedestrian_Location_2',
       'Pedestrian_Location_3', 'Pedestrian_Location_4',
       'Pedestrian_Location_5', 'Pedestrian_Location_6',
       'Pedestrian_Location_7', 'Pedestrian_Location_8',
       'Pedestrian_Location_9', 'Pedestrian_Location_10',
       'Pedestrian_Movement_0', 'Pedestrian_Movement_1',
       'Pedestrian_Movement_2', 'Pedestrian_Movement_3',
       'Pedestrian_Movement_4', 'Pedestrian_Movement_5',
       'Pedestrian_Movement_6', 

In [33]:
X = dataset.drop(["Age_of_Casualty", "Casualty_Severity"], axis=1)
y = dataset["Casualty_Severity"]
del dataset

In [34]:
print(X.shape)
print(y.shape)

(2216720, 72)
(2216720,)


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6357)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=6357)
del X, y
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1418700, 72)
(354676, 72)
(443344, 72)


In [36]:
X_train.describe()

,Casualty_Class_1,Casualty_Class_2,Casualty_Class_3,Sex_of_Casualty_1,Sex_of_Casualty_2,Age_Band_of_Casualty_1,Age_Band_of_Casualty_2,Age_Band_of_Casualty_3,Age_Band_of_Casualty_4,Age_Band_of_Casualty_5,...,Casualty_Type_20,Casualty_Type_21,Casualty_Type_22,Casualty_Type_23,Casualty_Type_90,Casualty_Type_97,Casualty_Type_98,Casualty_Home_Area_Type_1,Casualty_Home_Area_Type_2,Casualty_Home_Area_Type_3
count,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,...,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06,1.418700e+06
mean,6.312378e-01,2.445802e-01,1.241820e-01,5.828759e-01,4.168351e-01,1.806795e-02,2.882287e-02,4.725241e-02,1.467442e-01,1.270410e-01,...,2.788468e-03,5.515613e-03,1.607105e-04,4.229224e-06,4.754353e-03,8.669909e-05,2.748996e-05,6.728385e-01,8.092056e-02,1.004638e-01
std,4.824695e-01,4.298382e-01,3.297892e-01,4.930839e-01,4.930353e-01,1.331973e-01,1.673085e-01,2.121784e-01,3.538509e-01,3.330190e-01,...,5.273229e-02,7.406210e-02,1.267615e-02,2.056504e-03,6.878773e-02,9.310834e-03,5.243016e-03,4.691769e-01,2.727132e-01,3.006175e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
